Baremetal bluetooth dirver
---

This project is actually a bigger undertaking than I initially assumed it would be. Since I've made the decision to challenge myself by doing this from scratch, there is a lot of house keeping that needs to be done before we can even begin to make the driver. 

The first major hurdle that presented itself is that the STMCubeIDE does not have a serial monitor, meaning to get any printed feedback from the board requires that I write UART drivers. 

Second, since I am not relying on the RTOS or HAL, I also need to create my own data structures for managing data. 

Third, the implementation of the final driver and a simple program that allows you to remotely control which lights are being illuminated on the breadboard.

My end goal is to also make these libraries portable, so I can move between the F411RE and L4R4 architectures.

Once more, I followed the instruction presented on the study.embeddedexpert.io website for the baremetal bluetooth course, but to challenge myself, after the concepts videos I worked on the code by myself to completion before continuing to see where I had made mistakes.

I also rewrote major portions to keep with the goal of portability.

## Serial Monitor Output Driver

The STM32CubeIDE does not include a serial monitor by default. This code is intended to allow us to overcome this flaw and give us a way to communicate with the device.

The Nucleo F411RE board that I am using has tx/rx pins in the USB cable that we can use to monitor our board output with. These use alternate function 7 on pins PA2 (tx) and PA3 (rx).

Repository is available [here](https://github.com/terckert/baremetal-bluetooth.git)

### Goals
I have two types of nucleo boards and this needs to be portable between them. To this end:
1. It needs to be easily configurable
2. Only one instance of it needs to be active at any point in time
3. It needs to write to an attached console


### USART start up steps
1. Enable clock to TX pin
2. Enable clock to RX pin
3. Set TX to alternate function mode
4. Set TX alternate function register with mode
5. Set RX to alternate function mode
6. Set RX alternate function register with mode
7. Enable clock to UART
8. Disable UART module
9. Set UART baudrate
10. Set transfer direction
11. Enable interrupt in the NVIC
12. Enable UART module

### USART transmit function
1. Check that the transmit flag is in read state (1 = ready / 0 = not ready)
2. Place character to send in data register
3. Overwrite the put_char function for printf with new function


### Goals Resolution
For portability, I decided that a bunch of #defines at the top of the C file would make the most sense. These could be altered easily when porting into a project on another STM board.

### Final thoughts
There is a trade-off with readability and portability. Since the logic in the code uses the redefinitions exclusively, it forces the user to constant have to look away to find what a line of code actually does. I don't consider this a bad thing, as the user needs only configure the defines for their particular microprocessor/board to get full access to full access to the library.

## Circular data buffer

The end goal of an interrupt is to get out of it as quickly as possible. To this end, I needed to create a buffered data structure that would allow the interrupt to deposit the data as it became available and leave the handler. Data can then be processed elsewhere.

### Design of the the buffer
An array with two key components:
1. A head that points to the next stop where data can be inserted
2. A tail that tracks where the current reading position in the buffer is

A buffer is empty when the head and the tail are at the same position. We need to devise a formula that allows the pointers to roll over when they reach the end of the array to avoid read errors. $$ newIndex = (currentIndex + 1) \% arraySize $$ using this formula we can roll over safely when reaching the end of our alloted buffer. This also means that the tail, when not pointing to the same address as the head, points to a valid piece of data and the head always points to an empty spot.

One issue that presents itself is buffer overflow. Due to working on limited spaced hardware, I decided against using a true linked list and went with the array of fixed size. The push function returns 0 when a push is okay and a 1 when the final result of a push is a head and tail at the same position. While not elegant, it is useful for debug purposes.
